In [ ]:
# TODO: Need to load previous version of models.

import sys
import os
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/python-box')
sys.path.append('../input/ttach-kaggle')
# TODO: Add DETR for segmentation from hugginface
!pip uninstall transformers -y
# sys.path.append('../input/transformers-270')
# TODO: Add panoptic library offline later.
# !pip install python-box timm pytorch-lightning==1.4.0 grad-cam ttach
# !pip install git+https://github.com/cocodataset/panopticapi.git

In [ ]:
# Need to install hub first.
!pip install ../input/pawpularity-packages/huggingface_hub-0.1.2-py3-none-any.whl -f ./ --no-index
!pip install ../input/pawpularity-packages/transformers-4.12.5-py3-none-any.whl -f ./ --no-index
!pip install ../input/pawpularity-packages/panopticapi-0.1/panopticapi -f ./ --no-index
!pip install ../input/pawpularity-packages/timm-0.4.12-py3-none-any.whl -f ./ --no-index

In [ ]:
# !pip install git+https://github.com/cocodataset/panopticapi.git
# !pip install timm
# !pip install transformers --upgrade

In [ ]:
# !python -m pip download --only-binary :all: --dest . --no-cache transformers
# !python -m pip download --only-binary :all: --dest . --no-cache git+https://github.com/cocodataset/panopticapi.git

In [ ]:
# Detr needs timm but the src version isn't enough? Weird...
# Also we need the latest version of transformers but enable to make it work with the
# offline version of the library...

In [ ]:
# To avoid using internet
# TODO: Need to add the DETR models as well.
# Resnet to /root/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/timmswin/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22k.pth'
!cp '../input/timmswin/swin_tiny_patch4_window7_224.pth' '/root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth'
!cp '../input/pawpularity-packages/resnet50_ram-a26f946b.pth' '/root/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth'

# Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50_ram-a26f946b.pth" to /root/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth


In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset


import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule
from panopticapi.utils import rgb2id
from transformers import DetrFeatureExtractor, DetrForSegmentation, DetrConfig
from PIL import Image
import requests
import itertools
import seaborn as sns
import io
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import timm
warnings.filterwarnings("ignore")

# Segmentation model first

In [ ]:
DEBUG = False
RESIZE = (224, 224)
PETS_IDS = [18, 19]
TTA = True

if DEBUG:
    
    base_path = "../input/petfinder-pawpularity-score/train/{image_id}.jpg"
    csv_path = "../input/petfinder-pawpularity-score/train.csv"
    
else:
    base_path = "../input/petfinder-pawpularity-score/test/{image_id}.jpg"
    csv_path = "../input/petfinder-pawpularity-score/test.csv"    


device = "cuda:0" if torch.cuda.is_available() else "cpu"
config = DetrConfig.from_json_file("../input/pawpularity-packages/config.json")
detr_model_path = "../input/pawpularity-packages/d9b0227b10a6665774ec5240f8f850e135d711b0e0fad364a8b0bbca088ca298"
prerocessor_path = "../input/pawpularity-packages/preprocessor_config.json"

feature_extractor = DetrFeatureExtractor.from_pretrained(prerocessor_path)
# feature_extractor = feature_extractor.to(device)
model = DetrForSegmentation.from_pretrained(detr_model_path, config=config)
model = model.to(device)

def get_panoptic_information(img_path: str) -> None:
    # Try to resize the images.
    image = Image.open(img_path)
    image = image.resize(RESIZE)
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)
    outputs = model(**inputs)

    result = feature_extractor.post_process_panoptic(outputs, processed_sizes=[image.size])
    return result


def run_processing():
    df = pd.read_csv(csv_path)
    if DEBUG:
        df = df.sample(frac=0.01)
    image_ids = df["Id"].values.tolist()
    results = []
    for image_id in tqdm(image_ids):
        img_path = base_path.format(image_id=image_id)
        result = get_panoptic_information(img_path)
        results.extend(result)
    
    df = pd.DataFrame(results)
    df["Id"] = image_ids
    df.to_csv("panoptic_segmentation_test.csv", index=False)

In [ ]:
run_processing()
del model
del feature_extractor
import gc; gc.collect()

In [ ]:
!head -n 2 panoptic_segmentation_test.csv

In [ ]:
SEGMENTED_IDS = [18, 19]
class PetfinderDataset(Dataset):
    def __init__(self, df, image_size=224):
        self.metadata_df = pd.read_csv("panoptic_segmentation_test.csv")
        self.df = df
        self._X = df["path"].values
        self.ids = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([T.Resize([image_size, image_size]), T.PILToTensor()])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = Image.open(image_path)
        _id = self.ids[idx]
        panoptic_str = eval(self.metadata_df.loc[lambda df: df["Id"] == _id, "png_string"].values[0])
        panoptic_info_df = pd.DataFrame(eval(self.metadata_df.loc[lambda df: df["Id"] == _id, 
                                        "segments_info"].values[0]))

        if "category_id" in panoptic_info_df.columns:
            gdf = panoptic_info_df.groupby("category_id")["area"]
            # 18: dog, 19: cat, and other.
            areas_dict = gdf.sum().to_dict()
            counts_dict = gdf.count().to_dict()
            total_area = sum(areas_dict.values())
            total_count = sum(counts_dict.values())
            area_metafeatures = [areas_dict.get(k, 0) for k in PETS_IDS]
            area_metafeatures += [total_area - sum(area_metafeatures)]
            count_metafeatures = [counts_dict.get(k, 0) for k in PETS_IDS]
            count_metafeatures += [total_count - sum(count_metafeatures)]
            # Add width and height as metafeatures.
            area_metafeatures.append(image.size[0])
            area_metafeatures.append(image.size[1])
            metafeatures = torch.tensor(area_metafeatures + count_metafeatures, dtype=torch.float32)
        else:
            metafeatures = torch.tensor([0, 0, 0, 0, 0, 0, image.size[0], image.size[1]], dtype=torch.float32)
        image = self._transform(image)
        features = torch.tensor(self.df[FEATURES].values[idx],  dtype=torch.float32)
        if self._y is not None:
            label = self._y[idx]
            return image, label, metafeatures, features
        return image, metafeatures, features

class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        df,
        cfg,
    ):
        super().__init__()
        self._df = df
        self._cfg = cfg
    def test_dataloader(self):
        dataset = PetfinderDataset(self._df, self._cfg.transform.image_size)
        return DataLoader(dataset, **self._cfg.test_loader)
        
    def test_animals_dataloader(self):
        dataset = PetfinderDataset(self._df, "ANIMALS", self._cfg.transform.image_size)
        return DataLoader(dataset, **self._cfg.test_loader)
    
    def test_background_dataloader(self):
        dataset = PetfinderDataset(self._df, "BACKGROUND", self._cfg.transform.image_size)
        return DataLoader(dataset, **self._cfg.test_loader)

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform

In [ ]:
ROOT = "/kaggle/input/petfinder-pawpularity-score/"
SEED = 314
BATCH_SIZE = 4 # was 16.
config = {'seed': SEED,
          'root': ROOT, 
          'n_splits': 5,
          'epoch': 20,
          'trainer': {
              'gpus': 1,
              'accumulate_grad_batches': 1,
              'progress_bar_refresh_rate': 1,
              'fast_dev_run': False,
              'num_sanity_val_steps': 0,
              'resume_from_checkpoint': None,
          },
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'train_loader':{
              'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': BATCH_SIZE,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'test_loader': {
              'batch_size': BATCH_SIZE, 
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'model':{
              'name': 'swin_large_patch4_window7_224_in22k',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config = Box(config)

In [ ]:
META_FEATURES = ["cat_area", "dog_area", "background_area", "cat_count", "dog_count", "background_count", "width", "height"]
FEATURES = ["Subject Focus","Eyes","Face","Near","Action","Accessory","Group","Collage","Human","Occlusion","Info","Blur"]


class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self.transform = get_default_transforms()

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
        )
        self.meta_features = META_FEATURES
        self.features = FEATURES
        num_features = self.backbone.num_features + len(self.meta_features) + len(self.features)
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, num_features // 2)
        )
        self.fc2 = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features // 2, self.cfg.model.output_dim)
        )

    def forward(self, x, metafeatures, features):
        # TODO: Add more layers for the metafeatures later.
        x = self.backbone(x)
        metafeatures = torch.nn.functional.normalize(metafeatures)
        features = torch.nn.functional.normalize(features)
        x = torch.cat((x, metafeatures, features), dim=1).float()
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
df = pd.read_csv(os.path.join(config["root"], "test.csv"))
df["path"] = df["Id"].apply(lambda x: os.path.join(config.root, "test", x + ".jpg"))

In [ ]:
df.head()

In [ ]:
from pathlib import Path
models_folder = Path("/kaggle/input/swin-transformer-large/")
models = {}
# Will this work during inference? Might need to load one model at a time and predict then delete it.
# Trying with only one best fold.
for model_path in models_folder.glob("large*metafeatures*.ckpt"):
    model = Model(config)
    model_name = model_path.name
    model.load_state_dict(torch.load(model_path)['state_dict'])
    model = model.cuda().eval()
    models[model_name] = model

In [ ]:
len(models)

In [ ]:
assert len(models) == 5

In [ ]:
test_dataloader = PetfinderDataModule(df, config).test_dataloader()

In [ ]:
transform = get_default_transforms()["val"]

In [ ]:
import ttach as tta
tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90(angles=[90, 180, 270]),
    ]
)

In [ ]:
all_predictions = {}
alpha = 0.75
for model_name, model in models.items():
    tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)
    predictions = []
    for data, metafeatures, features in test_dataloader:
        metafeatures = metafeatures.cuda()
        features = features.cuda()
        data  = transform(data.cuda())
        # No TTA here since there is a bug...
        pred = model(data, metafeatures, features).sigmoid().detach().cpu() * 100
        if TTA:
            tta_pred = tta_model(data, metafeatures, features).sigmoid().detach().cpu() * 100
            weighted_pred = alpha * tta_pred + (1 - alpha) * pred
            predictions.extend(weighted_pred)
        else:
            predictions.extend(pred)
    del model
    del tta_model
    all_predictions[model_name] = predictions
    
del models
import gc
gc.collect()

In [ ]:
# New processing, to finish.

test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])

if meta:
    def size_and_shape(row):
        img = Image.open(row['path'])
        return pd.Series([img.size[0], img.size[1], img.size[1]/img.size[0], img.size[0]/img.size[1], os.path.getsize(row['path'])])

    scale_test = MinMaxScaler()

    test_df[['width', 'height', 'hw_ratio', 'wh_ratio', 'size']] = pd.DataFrame(scale_test.fit_transform(test_df.apply(size_and_shape, axis=1).values))

# Make the submission file and save it

In [ ]:
# TODO: Change to the length of the models.
predictions = np.array([np.array(predictions) for predictions in all_predictions.values()])

In [ ]:
submission_df = pd.read_csv(f'/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
submission_df['Pawpularity'] = np.mean(predictions, axis=0)
submission_df.to_csv('submission.csv', index = False)